# Notes

This jupyter notebook compares the results of a pipe loaded using the entire dataset of TNG100-3 in one go, vs the results when it is loaded in chunks. If these are identical, it will allow us to perform our calculations for larger TNG simulations.

From forum:

https://www.tng-project.org/data/forum/topic/203/loading-the-tng100-1-data/
https://www.tng-project.org/data/forum/topic/392/loading-subsample-of-all-particles-of-given-type/

# Imports

In [48]:
import h5py
import numpy as np
import illustris_python as il
from charlie_TNG_tools import temp2u
from astropy import constants as c
from astropy.cosmology import Planck15 as cosmosource


# Functions

In [2]:
def pSplitRange(indrange, numProcs, curProc, inclusive=False):
    """ Divide work for embarassingly parallel problems. 
    Accept a 2-tuple of [start,end] indices and return a new range subset.
    If inclusive==True, then assume the range subset will be used e.g. as input to snapshotSubseet(),
    which unlike numpy convention is inclusive in the indices."""
    assert len(indrange) == 2 and indrange[1] > indrange[0]

    if numProcs == 1:
        if curProc != 0:
            raise Exception("Only a single processor but requested curProc>0.")
        return indrange

    # split array into numProcs segments, and return the curProc'th segment
    splitSize = int(np.floor( (indrange[1]-indrange[0]) / numProcs ))
    start = indrange[0] + curProc*splitSize
    end   = indrange[0] + (curProc+1)*splitSize

    # for last split, make sure it takes any leftovers
    if curProc == numProcs-1:
        end = indrange[1]

    if inclusive and curProc < numProcs-1:
        # not for last split/final index, because this should be e.g. NumPart[0]-1 already
        end -= 1

    return [start,end]


def loadSubset(simPath, snap, partType, fields, chunkNum=0, totNumChunks=1):
    """ 
    Load part of a snapshot.
    frm Dylan Nelson: https://www.tng-project.org/data/forum/topic/203/loading-the-tng100-1-data/
    """
    nTypes = 6
    ptNum = il.util.partTypeNum(partType)

    with h5py.File(il.snapshot.snapPath(simPath,snap),'r') as f:
        numPartTot = il.snapshot.getNumPart( dict(f['Header'].attrs.items()) )[ptNum]

    # define index range
    indRange_fullSnap = [0,numPartTot-1]
    indRange = pSplitRange(indRange_fullSnap, totNumChunks, chunkNum, inclusive=True)

    # load a contiguous chunk by making a subset specification in analogy to the group ordered loads
    subset = { 'offsetType'  : np.zeros(nTypes, dtype='int64'),
               'lenType'     : np.zeros(nTypes, dtype='int64') }

    subset['offsetType'][ptNum] = indRange[0]
    subset['lenType'][ptNum]    = indRange[1]-indRange[0]+1

    # add snap offsets (as required)
    with h5py.File(il.snapshot.offsetPath(simPath,snap),'r') as f:
        subset['snapOffsets'] = np.transpose(f['FileOffsets/SnapByType'][()])

    # load from disk
    r = il.snapshot.loadSubset(simPath, snap, partType, fields, subset=subset)

    return r

# Test the TNG data subset loading functions

In [33]:
############
#Initialise#
############

#define constants for warm-phase gas mass fraction
T_h = 10**7  #hot phase gase temperature [Kelvin] 
T_c = 10**3  #cold phase gas temperature [Kelvin]
x_h = 0.75   #Hydrogen mass fraction

nSubLoads = 100
snap_number = 99 #snapshot number for test
sim_to_use = 'TNG100-3'
basePath = '/virgo/simulations/IllustrisTNG/{0}/output/'.format(sim_to_use)
fields=['Density',
        'ElectronAbundance',
        'StarFormationRate',
        'InternalEnergy',
        'Coordinates',
        'Masses',
        'SubfindDMDensity']

########################
########################
##load full simulation##
########################
########################

dataPT0 = il.snapshot.loadSubset(basePath, snap_number, 'gas', fields=fields)

######################################
#create warm-phase gas mass fractions#
######################################
#note: this is from _w_frac_new() function in 
#raven:/u/cwalker/Illustris_FRB_Project/oct2_2021_output/IGM_new_scripts/job_raven.py

density = dataPT0['Density'] #the density values along the light ray in gcm**-3
sfr     = dataPT0['StarFormationRate'] #the star formation rate along the light ray in g/s
ie      = dataPT0['InternalEnergy'] #the internal energy along the light ray in erg/g
ea      = dataPT0['ElectronAbundance'] #the electron abundance along the light ray

#calculate x and w, cold and warm phase gas mass fractions
x_frac = (temp2u(T_h,ea)-ie)/(temp2u(T_h,ea)-temp2u(T_c,ea)) #cold phase mass fraction
w_frac = 1 - x_frac # warm phase mass fraction

#only modify electron abundance if sfr = 0
w_frac[np.where(sfr==0)]=1

#append to the data dictionary
dataPT0['Warm'] = w_frac

#################################################################
#store data which will be checked against partial loading method#
#################################################################
totfull = dataPT0['Density'].shape[0] #total number of particles
totfull_warm = dataPT0['Warm']
print(totfull_warm.shape)

##################################################################################################
##################################################################################################
##load simulation on a partwise basis, compare number of particles to when loading full snapshot##
##################################################################################################
##################################################################################################

#initialise
totpart = 0 #initialise count of total number of particles
totpart_warm = []

#load partwise
for i in range(nSubLoads):
    data = loadSubset(basePath,snap_number, 'gas', fields,chunkNum=i, totNumChunks=nSubLoads)
    #create warm-phase gas mass fraction
    density = data['Density'] #the density values along the light ray in gcm**-3
    sfr     = data['StarFormationRate'] #the star formation rate along the light ray in g/s
    ie      = data['InternalEnergy'] #the internal energy along the light ray in erg/g
    ea      = data['ElectronAbundance'] #the electron abundance along the light ray
    #calculate x and w, cold and warm phase gas mass fractions
    x_frac = (temp2u(T_h,ea)-ie)/(temp2u(T_h,ea)-temp2u(T_c,ea)) #cold phase mass fraction
    w_frac = 1 - x_frac # warm phase mass fraction
    #only modify electron abundance if sfr = 0
    w_frac[np.where(sfr==0)]=1
    data['Warm']=w_frac
    #store data which will be checked against full loading method
    totpart+=data['Density'].shape[0] #tally number of particles
    totpart_warm.append(data['Warm'])



(88935326,)


In [34]:

#perform check on total number of particles
if totpart==totfull:
    print('CHECK 1 PASSED, BOTH LOADING TYPES RESULT IN {0} PARTICLES'.format(totpart))
else:
    print(totpart/2,totfull)
    print('CHECK 1 FAILED')




CHECK 1 PASSED, BOTH LOADING TYPES RESULT IN 88935326 PARTICLES


In [35]:
#convert partial arrays to form which can be compared to total
totpart_warm_flat = [item for sublist in totpart_warm for item in sublist]
totpart_warm_flat = np.array(totpart_warm_flat)

In [44]:
#perform check on warm-phase gas mass fraction values
test2 = (totpart_warm_flat==totfull_warm)
if False in test2:
    print('CHECK 2 FAILED: WARM-PHASE GAS MASS FRACTIONS ARE NOT IDENTICAL')
else:
    print('CHECK 2 PASSED')


CHECK 2 PASSED


# Test loading the same pipe and make sure the results are equal

In [49]:
############
#Initialise#
############

npipes      = 1  #number of pipes to create
snap_number = 99 #snapshot number for test

#define constants
T_h = 10**7  #hot phase gase temperature [Kelvin] 
T_c = 10**3  #cold phase gas temperature [Kelvin]
x_h = 0.75   #Hydrogen mass fraction

#simulation to use
sim_to_use = 'TNG100-3'
#base path to simulation
basePath = '/virgo/simulations/IllustrisTNG/{0}/output/'.format(sim_to_use)



#######################
#load whole simulation#
#######################

print('Loading Simulation: {0}'.format(sim_to_use))
#gas, i.e. partType0 data to load
dataPT0 = il.snapshot.loadSubset(basePath, snap_number, 'gas', fields=['Density',
                                                                       'ElectronAbundance',
                                                                       'StarFormationRate',
                                                                       'InternalEnergy',
                                                                       'Coordinates',
                                                                       'Masses',
                                                                       'SubfindDMDensity'])


######################################
#create warm-phase gas mass fractions#
######################################
#note: this is from _w_frac_new() function in 
#raven:/u/cwalker/Illustris_FRB_Project/oct2_2021_output/IGM_new_scripts/job_raven.py

density = dataPT0['Density'] #the density values along the light ray in gcm**-3
sfr     = dataPT0['StarFormationRate'] #the star formation rate along the light ray in g/s
ie      = dataPT0['InternalEnergy'] #the internal energy along the light ray in erg/g
ea      = dataPT0['ElectronAbundance'] #the electron abundance along the light ray


#calculate x and w, cold and warm phase gas mass fractions
x_frac = (temp2u(T_h,ea)-ie)/(temp2u(T_h,ea)-temp2u(T_c,ea)) #cold phase mass fraction
w_frac = 1 - x_frac # warm phase mass fraction

#only modify electron abundance if sfr = 0
w_frac[np.where(sfr==0)]=1

#append to the data dictionary
dataPT0['Warm'] = w_frac



#############
#load header#
#############

header = il.groupcat.loadHeader(basePath,snap_number)

#######################
#Initialise pipe stuff#
#######################

#The number of cells in the chosen snapshot
ncells = dataPT0['Coordinates'].shape[0]
print('Number of cells in snapshot {0} is {1}'.format(snap_number,ncells))

#The width of the pipe
pipe_width = 200 #By following zhang+20 definition, sides will be 200ckpc/h in length
print('Pipe width will be {0} ckpc/h'.format(pipe_width))

#The number of bins along a single line of sight
nbins=10000 #Zhang+20 definition: 10,000
print('There will be {0} bins on each sightline'.format(nbins))

#Define the mass of a proton for dDM/dz calculations
protonmass = c.m_p.to('kg')
print('Proton mass is {0}'.format(protonmass))

#Define the hydrogen mass fraction for dDM/dz calculations
hmassfrac = 3./4.
print('Chosen H mass fraction is {0}. Check whether this is correct'.format(hmassfrac))

#calculate the critical density at redshift zero for structure categorisation
#source to formula: https://astronomy.swin.edu.au/cosmos/c/Critical+Density
grav=c.G.to('m**3/(kg*s**2)') #g as a YT quantity in correct units
H=cosmosource.H(0).to('km/s/Mpc') #hubble const at z=0 in km/s/Mpc
my_dens_crit = ((3 * H**2)/(8*np.pi* grav)).to('kg/m**3')
print('Critical density at z=0 = {0}'.format(my_dens_crit))

Loading Simulation: TNG100-3
Number of cells in snapshot 99 is 88935326
Pipe width will be 200 ckpc/h
There will be 10000 bins on each sightline
Proton mass is 1.67262192369e-27 kg
Chosen H mass fraction is 0.75. Check whether this is correct
Critical density at z=0 = 8.619160453152573e-27 kg / m3


## Create the pipe details

In [50]:
#############
#Create Pipe#
#############



#########################################
#define los coordinates at start of pipe#
#########################################

#By Zhang+20 definition of following x-axis,
#x will be zero, y and z will be random
#units default = ckpc/h (compare box size to https://www.tng-project.org/about/)

pipe_start_coords = np.array([0,
                     np.random.uniform(0,header['BoxSize'],1)[0],
                     np.random.uniform(0,header['BoxSize'],1)[0]])
print('Random start cell coordinates: {0}'.format(pipe_start_coords))

###################################
#define coordinates at end of pipe#
###################################

#By Zhang+20 definition of following x-axis,
#x will be length of simulation,y and z will be same as start coords

pipe_end_coords = pipe_start_coords+np.array([header['BoxSize'],0,0])
print('Pipe end cell coordinates: {0}'.format(pipe_end_coords))

########################
#construct pipe corners#
########################

#Add and subtract half of pipe length from y and z coords for y and z boundaries
#code adapted from https://stackoverflow.com/questions/33540109/plot-surfaces-on-a-cube

c1s = pipe_start_coords + np.array([0,pipe_width/2,pipe_width/2]) #start corner 1
c2s = pipe_start_coords + np.array([0,-pipe_width/2,-pipe_width/2]) #start corner 2
c3s = pipe_start_coords + np.array([0,pipe_width/2,-pipe_width/2]) #start corner 3
c4s = pipe_start_coords + np.array([0,-pipe_width/2,pipe_width/2]) #start corner 4

c1e = pipe_end_coords + np.array([0,pipe_width/2,pipe_width/2]) #end corner 1
c2e = pipe_end_coords + np.array([0,-pipe_width/2,-pipe_width/2]) #end corner 2
c3e = pipe_end_coords + np.array([0,pipe_width/2,-pipe_width/2]) #end corner 3
c4e = pipe_end_coords + np.array([0,-pipe_width/2,pipe_width/2]) #end corner 4

corners = np.array([c1s,c2s,c3s,c4s,c1e,c2e,c3e,c4e])

Random start cell coordinates: [    0.         40997.32329197 60139.75065242]
Pipe end cell coordinates: [75000.         40997.32329197 60139.75065242]


## Get the data in this pipe using full snap loading method

In [52]:
###########################################
#get cells in this pipe from full data set#
###########################################

#adapted from https://stackoverflow.com/questions/42352622/finding-points-within-a-bounding-box-with-numpy
#I think this is right but if I get any strange results, double check the theory

yz_pts = dataPT0['Coordinates'][:,[1,2]] #all y and z coords
print('All y and z values: {0}'.format(yz_pts))

ur = c1s[1:] #upper right of pipe start (y and z only)
ll = c2e[1:] #lower left of pipe end (y and z only)
print('Upper right: {0}'.format(ur))
print('Lower left: {0}'.format(ll))

inidx = np.all((ll <= yz_pts) & (yz_pts <= ur), axis=1) #indexes of cells in pipe
print(inidx)

###########################
#get data of cells in pipe#
###########################

pipe_cell_coords = dataPT0['Coordinates'][inidx]       #coordinates [ckpc/h]
pipe_cell_dens   = dataPT0['Density'][inidx]           #densities [(1e10Msun/h)/(ckpc/h)**3]
pipe_cell_elab   = dataPT0['ElectronAbundance'][inidx] #electron abundance [-]
pipe_cell_sfr    = dataPT0['StarFormationRate'][inidx] #star formation rate [Msun/yr]
pipe_cell_dark   = dataPT0['SubfindDMDensity'][inidx]  #comoving dark matter density [(1e10Msun/h)/(ckpc/h)**3]
pipe_cell_warm   = dataPT0['Warm'][inidx]              #warm-phase gas mass fraction

print('{0} cells in this pipe'.format(dataPT0['Coordinates'][inidx].shape[0]))

All y and z values: [[26340.14797967 18286.47294083]
 [26333.73134483 18284.21846164]
 [26332.11977328 18286.97684438]
 ...
 [32512.26227586 41876.47687281]
 [32759.88182406 44010.40573232]
 [46529.37972099 49350.65699391]]
Upper right: [41097.32329197 60239.75065242]
Lower left: [40897.32329197 60039.75065242]
[False False False ... False False False]
491 cells in this pipe


## get the data in this pipe using partial snap loading method

In [60]:
###########################################
#get cells in this pipe by partial loading#
###########################################

#initialise
pipe_cell_coords_part=[]
pipe_cell_dens_part = []
pipe_cell_elab_part=[]
pipe_cell_sfr_part=[]
pipe_cell_dark_part = []
pipe_cell_warm_part=[]

#########################
#loop over partial loads#
#########################

for i in range(nSubLoads):
    
    ###########################
    #load the partial data set#
    ###########################
    
    data = loadSubset(basePath,snap_number, 'gas', fields,chunkNum=i, totNumChunks=nSubLoads)

    #####################################
    #create warm-phase gas mass fraction#
    #####################################
    
    density = data['Density'] #the density values along the light ray in gcm**-3
    sfr     = data['StarFormationRate'] #the star formation rate along the light ray in g/s
    ie      = data['InternalEnergy'] #the internal energy along the light ray in erg/g
    ea      = data['ElectronAbundance'] #the electron abundance along the light ray
    #calculate x and w, cold and warm phase gas mass fractions
    x_frac = (temp2u(T_h,ea)-ie)/(temp2u(T_h,ea)-temp2u(T_c,ea)) #cold phase mass fraction
    w_frac = 1 - x_frac # warm phase mass fraction
    #only modify electron abundance if sfr = 0
    w_frac[np.where(sfr==0)]=1
    data['Warm']=w_frac    

    ########################
    #get cells in this pipe#
    ########################
    
    yz_pts = data['Coordinates'][:,[1,2]]
    ur = c1s[1:] #upper right of pipe start (y and z only)
    ll = c2e[1:] #lower left of pipe end (y and z only)
    inidx = np.all((ll <= yz_pts) & (yz_pts <= ur), axis=1) #indexes of cells in pipe

    ###########################
    #get data of cells in pipe#
    ###########################

    pipe_cell_coords_part.append(data['Coordinates'][inidx])       #coordinates [ckpc/h]
    pipe_cell_dens_part.append(data['Density'][inidx])           #densities [(1e10Msun/h)/(ckpc/h)**3]
    pipe_cell_elab_part.append(data['ElectronAbundance'][inidx]) #electron abundance [-]
    pipe_cell_sfr_part.append(data['StarFormationRate'][inidx]) #star formation rate [Msun/yr]
    pipe_cell_dark_part.append(data['SubfindDMDensity'][inidx])  #comoving dark matter density [(1e10Msun/h)/(ckpc/h)**3]
    pipe_cell_warm_part.append(data['Warm'][inidx])

## flatten the partially loaded data

In [61]:
pipe_cell_coords_part_flat = np.array([item for sublist in pipe_cell_coords_part for item in sublist])
pipe_cell_dens_part_flat   = np.array([item for sublist in pipe_cell_dens_part for item in sublist])
pipe_cell_elab_part_flat   = np.array([item for sublist in pipe_cell_elab_part for item in sublist])
pipe_cell_sfr_part_flat    = np.array([item for sublist in pipe_cell_sfr_part for item in sublist])
pipe_cell_dark_part_flat   = np.array([item for sublist in pipe_cell_dark_part for item in sublist])
pipe_cell_warm_part_flat   = np.array([item for sublist in pipe_cell_warm_part for item in sublist])


## compare data loaded by full and partial methods

In [73]:
#perform check on warm-phase gas mass fraction values
coordstest = (pipe_cell_coords_part_flat==pipe_cell_coords)
if False in coordstest:
    print('COORDS CHECK FAILED: COORDINATES ARE NOT IDENTICAL')
else:
    print('COORDS CHECK PASSED')

denstest = (pipe_cell_dens_part_flat==pipe_cell_dens)
if False in denstest:
    print('DENS CHECK FAILED: DENSITIES ARE NOT IDENTICAL')
else:
    print('DENS CHECK PASSED')
    
elabtest = (pipe_cell_elab_part_flat==pipe_cell_elab)
if False in elabtest:
    print('ELAB CHECK FAILED: ELECTRONABUNDANCES ARE NOT IDENTICAL')
else:
    print('ELAB CHECK PASSED')
    
sfrtest = (pipe_cell_sfr_part_flat==pipe_cell_sfr)
if False in sfrtest:
    print('SFR CHECK FAILED: STAR FORMATION RATES ARE NOT IDENTICAL')
else:
    print('SFR CHECK PASSED')

darktest = (pipe_cell_dark_part_flat==pipe_cell_dark)
if False in darktest:
    print('DARK CHECK FAILED: DARK MATTER DENSITIES ARE NOT IDENTICAL')
else:
    print('DARK CHECK PASSED')
    
warmtest = (pipe_cell_warm_part_flat==pipe_cell_warm)
if False in warmtest:
    print('WARM CHECK FAILED: WARM-PHASE GAS MASS FRACTIONS ARE NOT IDENTICAL')
else:
    print('WARM CHECK PASSED')

COORDS CHECK PASSED
DENS CHECK PASSED
ELAB CHECK PASSED
SFR CHECK PASSED
DARK CHECK PASSED
WARM CHECK PASSED
